# Example 5: Extract Class, Move Field, Move Method

[Extract class in the refactoring catalog.](http://refactoring.com/catalog/extractClass.html)

[Move field in the refactoring catalog](http://refactoring.com/catalog/moveField.html).

[Move method in the refactoring catalog](http://refactoring.com/catalog/moveMethod.html).

- Do this when a class is responsible for too many things (> 5 methods, > 5 properties)
- Define a new class with all the fields you need to do the work.
- Implement `@property` and `@property.setter` to point at the new helper class so old usage continues to work.
- Move the methods that interact with these properties onto the interior class, instead of on the exterior class. Use `@property` to point existing usage at the new helper class.
- Use warnings to track down old usage. Move that usage to directly interact with the interior class. Remove all the `@property`s you used for indirection.

In [2]:
class Pet:
    def __init__(self, name):
        self.name = name

In [4]:
my_pet = Pet('Shadow')
print(my_pet.name)

Shadow


---

More complex

In [50]:
class Pet:
    def __init__(self, name, joined_family):
        self.name = name
        self.joined_family = joined_family

In [51]:
import datetime

In [52]:
adoption_date = datetime.date.today()
my_pet = Pet('Shadow', adoption_date)
print('%s was adopted on %s' % (my_pet.name, my_pet.joined_family))

Shadow was adopted on 2016-05-21


---

Now we want to know what kind of pet it is, so we add various attributes and helper methods.

In [53]:
class Pet:
    def __init__(self, name, joined_family, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        self.name = name
        self.joined_family = joined_family
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

    @property
    def is_mammal(self):
        return (
            self.drinks_milk and
            not self.has_scales and
            not self.lays_eggs)

In [54]:
my_pet = Pet('Shadow', adoption_date, drinks_milk=True)
print('%s is a mammal? %s' % (my_pet.name, my_pet.is_mammal))

Shadow is a mammal? True


---

Maybe this class has too many responsibilities. We should separate the details of what the animal is from what we know about the pet.

In [55]:
import warnings

In [73]:
class Animal:
    def __init__(self, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

class Pet:
    def __init__(self, name, joined_family, animal=None, **kwargs):
        self.name = name
        self.joined_family = joined_family

        if kwargs:
            warnings.warn('Must directly pass an Animal instance', DeprecationWarning)
            animal = Animal(**kwargs)
        elif animal is None:
            raise TypeError('Must supply "animal"')
        
        self.animal = animal

    @property
    def has_scales(self):
        warnings.warn('Use animal.has_scales instead', DeprecationWarning)
        return self.animal.has_scales

    @property
    def lays_eggs(self):
        warnings.warn('Use animal.lays_eggs instead', DeprecationWarning)
        return self.animal.lays_eggs

    @property
    def drinks_milk(self):
        warnings.warn('Use animal.drinks_milk instead', DeprecationWarning)
        return self.animal.drinks_milk

    @property
    def is_mammal(self):
        return (
            self.animal.drinks_milk and
            not self.animal.has_scales and
            not self.animal.lays_eggs)

---

Old usage keeps working

In [75]:
my_pet = Pet('Shadow', adoption_date, drinks_milk=True)  # Warning expected
print('%s is a mammal? %s' % (my_pet.name, my_pet.is_mammal))

Shadow is a mammal? True


/Users/bslatkin/projects/pycon2016/lib/python3.5/site-packages/ipykernel/__main__.py:13: DeprecationWarning: Must directly pass an Animal instance


---

Missing parameters for new usage will break like you called the function incorrectly.

In [76]:
my_pet = Pet('Shadow', adoption_date)  # Error expected

TypeError: Must supply "animal"

---

New usage works and no warnings.

In [78]:
animal = Animal(drinks_milk=True)
my_pet = Pet('Shadow', adoption_date, animal)
print('%s is a mammal? %s' % (my_pet.name, my_pet.is_mammal))

Shadow is a mammal? True


---

Now assume that all of the references have been moved to the new version.

In [70]:
class Animal:
    def __init__(self, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

class Pet:
    def __init__(self, name, joined_family, animal):
        self.name = name
        self.joined_family = joined_family
        self.animal = animal

    @property
    def is_mammal(self):
        return (
            self.animal.drinks_milk and
            not self.animal.has_scales and
            not self.animal.lays_eggs)

In [71]:
animal = Animal(drinks_milk=True)
my_pet = Pet('Shadow', adoption_date, animal)
print('%s is a mammal? %s' % (my_pet.name, my_pet.is_mammal))

Shadow is a mammal? True


---

Now we want to move the `is_mammal` method into the `Animal` class. It's best not to refactor too many things at the same time. The point here is you can do it piecemeal and you'll existing a transitional state where part of your APIs looks good and part of it doesn't look good yet.

In [79]:
class Animal:
    def __init__(self, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

    @property
    def is_mammal(self):
        return (
            self.drinks_milk and
            not self.has_scales and
            not self.lays_eggs)
        
class Pet:
    def __init__(self, name, joined_family, animal):
        self.name = name
        self.joined_family = joined_family
        self.animal = animal

    @property
    def is_mammal(self):
        warnings.warn('Must use animal.is_mammal')
        return self.animal.is_mammal

In [81]:
animal = Animal(drinks_milk=True)
my_pet = Pet('Shadow', adoption_date, animal)
print('%s is a mammal? %s' % (my_pet.name, my_pet.is_mammal))  # Warning expect

Shadow is a mammal? True


/Users/bslatkin/projects/pycon2016/lib/python3.5/site-packages/ipykernel/__main__.py:22: UserWarning: Must use animal.is_mammal


Fix all of the usages

In [82]:
class Animal:
    def __init__(self, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

    @property
    def is_mammal(self):
        return (
            self.drinks_milk and
            not self.has_scales and
            not self.lays_eggs)
        
class Pet:
    def __init__(self, name, joined_family, animal):
        self.name = name
        self.joined_family = joined_family
        self.animal = animal

Old usage stops working

In [86]:
animal = Animal(drinks_milk=True)
my_pet = Pet('Shadow', adoption_date, animal)
print('%s is a mammal? %s' % (my_pet.name, my_pet.is_mammal))

AttributeError: 'Pet' object has no attribute 'is_mammal'

New usage works without any problems.

In [84]:
animal = Animal(drinks_milk=True)
my_pet = Pet('Shadow', adoption_date, animal)
print('%s is a mammal? %s' % (my_pet.name, my_pet.animal.is_mammal))

Shadow is a mammal? True
